In [4]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import load_dataset

ds = load_dataset("kmack/Phishing_urls")

In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 567056
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 70882
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 70882
    })
})

In [3]:
train_df = ds['train'].to_pandas()

In [4]:
test_df = ds['test'].to_pandas()
valid_df = ds['valid'].to_pandas()

In [5]:
print(train_df.head(),test_df.head(),valid_df.head())

                                                text  label
0             xenophongroup.com/montjoie/compgns.htm      0
1    www.azzali.eu/&usg=AOvVaw2phVSb_ENMrkATGNx5LQ0l      1
2                     guildmusic.edu.au/js/index.htm      1
3  memo.unexpectedrunner.com/ezxgytw4et\nholotili...      1
4  en.wikipedia.org/wiki/Category:American_televi...      0                                                 text  label
0    www.linuxplanet.com/linuxplanet/reviews/4149/1/      1
1  sdnmargorejo1-403.sch.id/images/login.alibaba....      1
2  '9d345009-a-62cb3a1a-s-sites.googlegroups.com/...      0
3            everyculture.com/Ma-Ni/New-Zealand.html      0
4                    www.aoseocn.acseosn.selfie.ltd/      1                                                 text  label
0       absoluteastronomy.com/topics/Sri_Lankan_Navy      0
1                    www.angelfire.com/amiga/grotto/      1
2  www.etc-meisai.jp.lmxnzp.shop/kaduxn.php?lia71...      1
3  http://torcache.net/torrent/DCA42EC92

verifying data leakage prevention

In [6]:
import pandas as pd

# assuming you have 'id' or can hash rows
train_hashes = set(train_df.apply(lambda x: hash(tuple(x)), axis=1))
test_hashes = set(test_df.apply(lambda x: hash(tuple(x)), axis=1))
valid_hashes = set(valid_df.apply(lambda x: hash(tuple(x)), axis=1))

print("Train-Test overlap:", len(train_hashes & test_hashes))
print("Train-Valid overlap:", len(train_hashes & valid_hashes))
print("Test-Valid overlap:", len(test_hashes & valid_hashes))

Train-Test overlap: 0
Train-Valid overlap: 0
Test-Valid overlap: 0


In [7]:
print("Shape of the training dataset is :",train_df.shape)
print("No. of missing values :",train_df.isnull().sum())
print("Duplicate values in urls:",train_df["text"].duplicated().sum())
print("Number of unique values : ",train_df.nunique())

Shape of the training dataset is : (567056, 2)
No. of missing values : text     0
label    0
dtype: int64
Duplicate values in urls: 31218
Number of unique values :  text     535838
label         2
dtype: int64


In [8]:
train_df = train_df.drop_duplicates(subset = "text")

In [9]:
print("Duplicate values in urls:",train_df["text"].duplicated().sum())

Duplicate values in urls: 0


Tokenization

In [10]:
import re

def clean_url(url):
    url = url.lower().strip()

    # Remove protocol but keep subdomains
    url = re.sub(r'^https?://', '', url)

    # Remove trailing slashes and fragments (#, etc.)
    url = re.sub(r'[#]+.*', '', url)
    
    # Remove common tracking parameters (optional)
    url = re.sub(r'(\?|&)(utm_[^=]+|fbclid|gclid)=[^&]+', '', url)

    # Remove non-useful symbols, but keep . / ? = _ - &
    url = re.sub(r'[^a-z0-9./?=&_-]', '', url)

    # Replace multiple slashes with one (to avoid //)
    url = re.sub(r'/+', '/', url)

    return url


In [11]:
train_df['text'] = train_df['text'].apply(clean_url)
valid_df['text'] = valid_df['text'].apply(clean_url)
test_df['text'] = test_df['text'].apply(clean_url)

In [12]:
train_df.shape

(535838, 2)

In [13]:
train_df.duplicated().sum()

np.int64(9575)

In [14]:
train_df.shape

(535838, 2)

In [15]:
train_df = train_df.drop_duplicates(subset = "text")

In [16]:
train_df.duplicated().sum()

np.int64(0)

In [17]:
url_lengths = train_df['text'].apply(len)
print(url_lengths.describe())


count    525364.000000
mean         46.736122
std          46.452726
min           0.000000
25%          21.000000
50%          34.000000
75%          56.000000
max        2307.000000
Name: text, dtype: float64


In [18]:
import numpy as np
maxlen = int(np.percentile(url_lengths, 98))
print("Chosen maxlen:", maxlen)


Chosen maxlen: 182


In [19]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Build a character-level tokenizer
tokenizer = Tokenizer(char_level=True, lower=True, filters='')  # keep special chars
tokenizer.fit_on_texts(train_df['text'])

# Convert to sequences
X_train_seq = tokenizer.texts_to_sequences(train_df['text'])
X_val_seq   = tokenizer.texts_to_sequences(valid_df['text'])
X_test_seq  = tokenizer.texts_to_sequences(test_df['text'])

# Padding to a fixed length
maxlen = maxlen  # adjust based on URL length distribution
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=maxlen, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=maxlen, padding='post')

# Labels
y_train = train_df['label'].values
y_val   = valid_df['label'].values
y_test  = test_df['label'].values


In [35]:
print("Vocab size:", len(tokenizer.word_index) + 1)
print("Sample tokens:", list(tokenizer.word_index.keys())[:50])


Vocab size: 44
Sample tokens: ['e', 'o', 'a', 'c', '.', 'i', 't', 's', 'n', '/', 'r', 'm', 'l', 'd', 'p', 'w', '-', 'u', 'h', 'b', 'g', 'f', '1', '0', '2', 'y', 'k', '3', '8', 'v', '5', '4', '9', '6', '7', 'x', 'j', '_', '=', 'z', 'q', '?', '&']


In [22]:
train_df

,text,label
0,xenophongroup.com/montjoie/compgns.htm,0
1,www.azzali.eu/&usg=aovvaw2phvsb_enmrkatgnx5lq0l,1
2,guildmusic.edu.au/js/index.htm,1
3,memo.unexpectedrunner.com/ezxgytw4etnholotilic...,1
4,en.wikipedia.org/wiki/categoryamerican_televis...,0
...,...,...
567049,ucdavisaggies.com/sports/m-footbl/mtt/allen_dr...,0
567050,www.aoscuu-smaocmeouusnauu.fhoarcu.museum.mw/,1
567051,www.en.wikipedia.org/wiki/charles_taylor_philo...,0
567054,revenue.ky.gov,0


In [23]:
X_train_pad.shape

(525364, 182)

In [25]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [36]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Concatenate, \
                                     BatchNormalization, ReLU, Bidirectional, LSTM, Dense, GlobalMaxPool1D
from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from copy import deepcopy


In [37]:
# Use your values
vocab_size = 44         # given
maxlen = 182            # given

# Model hyperparams (sensible defaults for URL data)
embedding_dim = 100      # smaller -> faster; increase to 100-200 if GPU and memory allow
conv_filters = 128
conv_kernel_sizes = [3,5,7]   # parallel convs capture tri/5/7-gram char patterns
pool_size = 2
bilstm_units = 128
dense_proj_units = 128   # final neural feature dimension to feed XGBoost

# Federated training hyperparams
num_clients = 10
federated_rounds = 5
local_epochs = 1        # keep small for FL experiments; increase later
batch_size = 256
learning_rate = 1e-3


In [38]:
def build_feature_extractor(vocab_size, maxlen,
                            embedding_dim=64,
                            conv_filters=128,
                            conv_kernel_sizes=[3,5,7],
                            pool_size=2,
                            bilstm_units=128,
                            dense_proj_units=128):
    inp = Input(shape=(maxlen,), dtype='int32', name='input_ids')
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                  input_length=maxlen, mask_zero=False, name='embedding')(inp)
    # Parallel convs
    convs = []
    for k in conv_kernel_sizes:
        c = Conv1D(filters=conv_filters, kernel_size=k, padding='same')(x)
        c = BatchNormalization()(c)
        c = ReLU()(c)
        c = MaxPooling1D(pool_size=pool_size)(c)
        convs.append(c)
    x = Concatenate(axis=-1)(convs)  # shape: (batch, reduced_len, conv_filters*len(k))
    # Optional extra conv to mix channels
    x = Conv1D(filters=conv_filters, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # BiLSTM for context; return_sequences=False to get a single vector per sample
    x = Bidirectional(LSTM(bilstm_units, return_sequences=False))(x)

    # Projection / fusion: compact vector for XGBoost. No final activation here.
    x = Dense(dense_proj_units, activation='relu', name='dense_proj')(x)
    # Note: don't apply dropout here because we want deterministic features for XGBoost
    model = Model(inputs=inp, outputs=x, name='cnn_bilstm_extractor')
    return model

# Build the model
feature_extractor = build_feature_extractor(vocab_size=vocab_size,
                                            maxlen=maxlen,
                                            embedding_dim=embedding_dim,
                                            conv_filters=conv_filters,
                                            conv_kernel_sizes=conv_kernel_sizes,
                                            pool_size=pool_size,
                                            bilstm_units=bilstm_units,
                                            dense_proj_units=dense_proj_units)
feature_extractor.summary()


c:\Users\LENOVO\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "cnn_bilstm_extractor"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_ids           │ (None, 182)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 182, 100)  │      4,400 │ input_ids[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_12 (Conv1D)  │ (None, 182, 128)  │     38,528 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_13 (Conv1D)  │ (None, 182, 128)  │     64,128 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_14 (Conv1D)  │ (None, 182, 128)  │     89,728 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 182, 128)  │        512 │ conv1d_12[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 182, 128)  │        512 │ conv1d_13[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 182, 128)  │        512 │ conv1d_14[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_12 (ReLU)     │ (None, 182, 128)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_13 (ReLU)     │ (None, 182, 128)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_14 (ReLU)     │ (None, 182, 128)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_9     │ (None, 91, 128)   │          0 │ re_lu_12[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_10    │ (None, 91, 128)   │          0 │ re_lu_13[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_11    │ (None, 91, 128)   │          0 │ re_lu_14[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 91, 384)   │          0 │ max_pooling1d_9[… │
│ (Concatenate)       │                   │            │ max_pooling1d_10… │
│                     │                   │            │ max_pooling1d_11… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_15 (Conv1D)  │ (None, 91, 128)   │    147,584 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 91, 128)   │        512 │ conv1d_15[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_15 (ReLU)     │ (None, 91, 128)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_3     │ (None, 256)       │    263,168 │ re_lu_15[0][0]    │
│ (Bidirectional)     │                   │            │                 

 Total params: 642,480 (2.45 MB)

 Trainable params: 641,456 (2.45 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [31]:
feature_extractor.summary()

Model: "cnn_bilstm_extractor"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_ids           │ (None, 182)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 182, 100)  │      4,400 │ input_ids[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 182, 128)  │     38,528 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 182, 128)  │     64,128 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 182, 128)  │     89,728 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 182, 128)  │        512 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 182, 128)  │        512 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 182, 128)  │        512 │ conv1d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 182, 128)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 182, 128)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 182, 128)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 91, 128)   │          0 │ re_lu[0][0]       │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 91, 128)   │          0 │ re_lu_1[0][0]     │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_2     │ (None, 91, 128)   │          0 │ re_lu_2[0][0]     │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 91, 384)   │          0 │ max_pooling1d[0]… │
│ (Concatenate)       │                   │            │ max_pooling1d_1[… │
│                     │                   │            │ max_pooling1d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 91, 128)   │    147,584 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 91, 128)   │        512 │ conv1d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 91, 128)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 256)       │    263,168 │ re_lu_3[0][0]     │
│ (Bidirectional)     │                   │            │                 

 Total params: 642,480 (2.45 MB)

 Trainable params: 641,456 (2.45 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [27]:
def compile_local_model(model, lr=1e-3):
    # Local training uses a temporary head for supervised training.
    # We'll attach a tiny classifier head for local training (binary classification),
    # then remove it for feature extraction.
    inp = model.input
    features = model.output
    out = Dense(1, activation='sigmoid', name='tmp_output')(features)
    train_model = Model(inputs=inp, outputs=out)
    train_model.compile(optimizer=Adam(lr), loss='binary_crossentropy', metrics=['accuracy'])
    return train_model

def get_model_weights(model):
    return model.get_weights()

def set_model_weights(model, weights):
    model.set_weights(weights)

def average_weights(weight_list):
    # simple element-wise average
    avg = []
    for weights in zip(*weight_list):
        avg.append(np.mean(weights, axis=0))
    return avg


In [28]:
def make_clients(X, y, num_clients):
    # simple IID split
    n = X.shape[0]
    indices = np.arange(n)
    np.random.shuffle(indices)
    splits = np.array_split(indices, num_clients)
    clients = [(X[s], y[s]) for s in splits]
    return clients

def federated_train(global_model, X_train, y_train,
                    num_clients=10,
                    rounds=5,
                    local_epochs=1,
                    batch_size=256,
                    lr=1e-3):
    # compile a local copy model with a temporary classifier head
    clients = make_clients(X_train, y_train, num_clients)
    # We'll hold global weights
    global_weights = get_model_weights(global_model)
    for r in range(rounds):
        print(f"\n=== Federated round {r+1}/{rounds} ===")
        local_weights = []
        for i, (X_c, y_c) in enumerate(clients):
            # create a fresh local train model from global extractor
            local_extractor = build_feature_extractor(vocab_size, maxlen,
                                                      embedding_dim, conv_filters,
                                                      conv_kernel_sizes, pool_size,
                                                      bilstm_units, dense_proj_units)
            # set global weights into local extractor
            set_model_weights(local_extractor, global_weights)
            # attach tiny head and compile
            local_train_model = compile_local_model(local_extractor, lr=lr)
            print(f" Client {i+1}: training on {X_c.shape[0]} samples")
            # Local training (small epochs)
            local_train_model.fit(X_c, y_c, epochs=local_epochs, batch_size=batch_size, verbose=1)
            # Extract updated extractor weights (strip off the tmp head)
            local_weights.append(get_model_weights(local_extractor))
            tf.keras.backend.clear_session()
        # Aggregate weights via FedAvg
        global_weights = average_weights(local_weights)
        # Set new global weights
        set_model_weights(global_model, global_weights)
        print(" Aggregated global weights updated.")
    return global_model

# Example usage:
# global_extractor = build_feature_extractor(...)
# trained_global_extractor = federated_train(global_extractor, X_train, y_train, ...)


In [ ]:
# assume 'trained_global_extractor' is the returned model from federated_train
# or you can skip federated and just use the initial feature_extractor

def extract_features(extractor_model, X):
    # extractor_model outputs the dense feature vector (dense_proj)
    features = extract  or_model.predict(X, batch_size=512, verbose=1)
    return features


In [30]:
# Run federated training
trained_global_extractor = federated_train(
    global_model=feature_extractor,
    X_train=X_train_pad,
    y_train=y_train,
    num_clients=3,        # you can reduce to 5 to test faster
    rounds=3,              # start small; increase later
    local_epochs=2,
    batch_size=256,
    lr=1e-3
)




=== Federated round 1/3 ===


c:\Users\LENOVO\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


 Client 1: training on 175122 samples
Epoch 1/2
  8/685 ━━━━━━━━━━━━━━━━━━━━ 7:24 656ms/step - accuracy: 0.5432 - loss: 0.6937

KeyboardInterrupt: 

In [33]:
# Extract features
X_train_feats = extract_features(trained_global_extractor, X_train_pad)
X_val_feats   = extract_features(trained_global_extractor, X_val_pad)


1027/1027 ━━━━━━━━━━━━━━━━━━━━ 434s 422ms/step
139/139 ━━━━━━━━━━━━━━━━━━━━ 56s 399ms/step


In [100]:
pip install -U xgboost


In [34]:
# Example: after federated training
# trained_global_extractor = federated_train(feature_extractor, X_train, y_train, ...)
# In case federated round is expensive, you can first try without FL:
# trained_global_extractor = feature_extractor  # pre-trained or untrained baseline

# 1) Extract features
# X_train_feats = extract_features(trained_global_extractor, X_train)         # shape (N, dense_proj_units)
# X_val_feats   = extract_features(trained_global_extractor, X_val_pad)      # your provided X_val_pad

# 2) Train XGBoost
xgb = XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    tree_method='hist'  # change to 'gpu_hist' on GPU
)

# Use early stopping on validation features
xgb.fit(
    X_train_feats, y_train,
    
    eval_set=[(X_val_feats, y_val)],
    verbose=True
)

c:\Users\LENOVO\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [12:01:22] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[0]	validation_0-logloss:0.65953
[1]	validation_0-logloss:0.62916
[2]	validation_0-logloss:0.60168
[3]	validation_0-logloss:0.57658
[4]	validation_0-logloss:0.55371
[5]	validation_0-logloss:0.53275
[6]	validation_0-logloss:0.51356
[7]	validation_0-logloss:0.49589
[8]	validation_0-logloss:0.47964
[9]	validation_0-logloss:0.46463
[10]	validation_0-logloss:0.45075
[11]	validation_0-logloss:0.43794
[12]	validation_0-logloss:0.42604
[13]	validation_0-logloss:0.41500
[14]	validation_0-logloss:0.40475
[15]	validation_0-logloss:0.39523
[16]	validation_0-logloss:0.38638
[17]	validation_0-logloss:0.37816
[18]	validation_0-logloss:0.37051
[19]	validation_0-logloss:0.36335
[20]	validation_0-logloss:0.35670
[21]	validation_0-logloss:0.35048
[22]	validation_0-logloss:0.34467
[23]	validation_0-logloss:0.33924
[24]	validation_0-logloss:0.33418
[25]	validation_0-logloss:0.32946
[26]	validation_0-logloss:0.32505
[27]	validation_0-logloss:0.32091
[28]	validation_0-logloss:0.31704
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, ...)

In [35]:
from sklearn.metrics import classification_report, roc_auc_score , accuracy_score

In [37]:
y_val_pred_proba = xgb.predict_proba(X_val_feats)[:,1]
y_val_pred = (y_val_pred_proba >= 0.5).astype(int)

print(classification_report(y_val, y_val_pred))
print("AUC:", roc_auc_score(y_val, y_val_pred_proba))


              precision    recall  f1-score   support

           0       0.92      0.84      0.88     35358
           1       0.85      0.93      0.89     35524

    accuracy                           0.89     70882
   macro avg       0.89      0.89      0.89     70882
weighted avg       0.89      0.89      0.89     70882

AUC: 0.9558220754737495


In [38]:
X_test_feats   = extract_features(trained_global_extractor, X_test_pad)

139/139 ━━━━━━━━━━━━━━━━━━━━ 46s 334ms/step


In [39]:
y_test_pred_proba = xgb.predict_proba(X_test_feats)[:,1]
y_test_pred = (y_test_pred_proba >= 0.5).astype(int)

print(classification_report(y_test, y_test_pred))
print("AUC:", roc_auc_score(y_test, y_test_pred_proba))

              precision    recall  f1-score   support

           0       0.92      0.84      0.88     35239
           1       0.85      0.93      0.89     35643

    accuracy                           0.89     70882
   macro avg       0.89      0.89      0.89     70882
weighted avg       0.89      0.89      0.89     70882

AUC: 0.9554982545126018


In [2]:
from sklearn.metrics import accuracy_score

In [ ]:
print(accuracy_score(y_test, y_test_pred))

NameError: name 'y_test' is not defined

WITH ATTENTION LAYER


In [123]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.layers import (Input, Embedding, Conv1D, BatchNormalization,
                                     LeakyReLU, MaxPooling1D, Concatenate,
                                     Bidirectional, LSTM, Dense, Dropout, Attention)
from tensorflow.keras.optimizers import Adam


In [127]:
from tensorflow.keras.layers import GlobalAveragePooling1D

def build_attention_model(vocab_size, maxlen,
                          embedding_dim=64,
                          conv_filters=128,
                          conv_kernel_sizes=[3,5,7],
                          pool_size=2,
                          bilstm_units=128,
                          dense_proj_units=128,
                          dropout_rate=0.3):

    inp = Input(shape=(maxlen,), dtype='int32', name='input_ids')

    # Embedding layer
    x = Embedding(input_dim=vocab_size + 1,
                  output_dim=embedding_dim,
                  input_length=maxlen,
                  mask_zero=False,
                  name='embedding')(inp)

    # Parallel Conv1D layers
    convs = []
    for k in conv_kernel_sizes:
        c = Conv1D(filters=conv_filters, kernel_size=k, padding='same')(x)
        c = BatchNormalization()(c)
        c = LeakyReLU(alpha=0.1)(c)
        c = MaxPooling1D(pool_size=pool_size)(c)
        convs.append(c)

    x = Concatenate(axis=-1)(convs)

    # Extra convolution for richer features
    x = Conv1D(filters=conv_filters, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)

    # BiLSTM contextual layer
    x = Bidirectional(LSTM(bilstm_units, return_sequences=True))(x)

    # Attention layer (self-attention)
    attn_out = Attention()([x, x])
    attn_vec = GlobalAveragePooling1D()(attn_out)  # safer than tf.reduce_mean

    # Dense + dropout
    x = Dense(dense_proj_units, activation='gelu')(attn_vec)
    x = Dropout(dropout_rate)(x)

    # Final sigmoid output
    out = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inp, outputs=out, name='cnn_bilstm_attention')
    return model


In [ ]:
def compile_local_model_from_global(global_model, learning_rate=0.001):
    """Clone and compile the global model for each client."""
    model = clone_model(global_model)
    model.set_weights(global_model.get_weights())
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


def average_weights(client_weights):
    """Average weights from all clients."""
    avg_weights = []
    for weights in zip(*client_weights):
        avg_weights.append(np.mean(weights, axis=0))
    return avg_weights


def federated_train(global_model, X_train, y_train,
                    X_val, y_val,
                    num_clients=5,
                    rounds=3,
                    local_epochs=1,
                    batch_size=256,
                    learning_rate=0.001,
                    non_iid=False):
    """Simulate federated averaging across clients."""

    n_samples = len(X_train)
    indices = np.arange(n_samples)
    np.random.shuffle(indices)

    client_splits = np.array_split(indices, num_clients)

    for r in range(rounds):
        print(f"\n🌍 Federated Round {r+1}/{rounds}")
        client_weights = []

        for i, idxs in enumerate(client_splits):
            print(f"  🔹 Client {i+1}: {len(idxs)} samples")

            X_c, y_c = X_train[idxs], y_train[idxs]

            local_model = compile_local_model_from_global(global_model, learning_rate)
            local_model.fit(X_c, y_c,
                            epochs=local_epochs,
                            batch_size=batch_size,
                            verbose=1)
            client_weights.append(local_model.get_weights())

        # Federated averaging
        new_weights = average_weights(client_weights)
        global_model.set_weights(new_weights)

        # Validation performance
        loss, acc = global_model.evaluate(X_val, y_val, verbose=1)
        print(f"  ✅ Global model accuracy after round {r+1}: {acc:.4f}")

    return global_model


In [131]:
# Example params (replace with your actual data vars)
vocab_size = 44
maxlen = 182
embedding_dim = 64
conv_filters = 128
bilstm_units = 128
dense_proj_units = 128
batch_size = 512
learning_rate = 0.001

# Build model
global_model = build_attention_model(vocab_size, maxlen,
                                     embedding_dim,
                                     conv_filters,
                                     bilstm_units=bilstm_units,
                                     dense_proj_units=dense_proj_units)

global_model.compile(optimizer=Adam(learning_rate=learning_rate),
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

# Run federated training (simulation)
trained_global = federated_train(global_model,
                                 X_train_pad, y_train,
                                 X_val=X_val_pad, y_val=y_val,
                                 num_clients=4,
                                 rounds=3,
                                 local_epochs=1,
                                 batch_size=batch_size,
                                 learning_rate=learning_rate)



🌍 Federated Round 1/3
  🔹 Client 1: 131341 samples
257/257 ━━━━━━━━━━━━━━━━━━━━ 373s 1s/step - accuracy: 0.8319 - loss: 0.3764
  🔹 Client 2: 131341 samples
257/257 ━━━━━━━━━━━━━━━━━━━━ 358s 1s/step - accuracy: 0.8378 - loss: 0.3674
  🔹 Client 3: 131341 samples
257/257 ━━━━━━━━━━━━━━━━━━━━ 392s 1s/step - accuracy: 0.8307 - loss: 0.3761
  🔹 Client 4: 131341 samples
257/257 ━━━━━━━━━━━━━━━━━━━━ 335s 1s/step - accuracy: 0.8306 - loss: 0.3783
  ✅ Global model accuracy after round 1: 0.4988

🌍 Federated Round 2/3
  🔹 Client 1: 131341 samples
257/257 ━━━━━━━━━━━━━━━━━━━━ 349s 1s/step - accuracy: 0.8699 - loss: 0.3109
  🔹 Client 2: 131341 samples
257/257 ━━━━━━━━━━━━━━━━━━━━ 354s 1s/step - accuracy: 0.8717 - loss: 0.3093
  🔹 Client 3: 131341 samples
257/257 ━━━━━━━━━━━━━━━━━━━━ 411s 2s/step - accuracy: 0.8692 - loss: 0.3142
  🔹 Client 4: 131341 samples
257/257 ━━━━━━━━━━━━━━━━━━━━ 394s 1s/step - accuracy: 0.8695 - loss: 0.3139
  ✅ Global model accuracy after round 2: 0.8057

🌍 Federated Round

In [132]:
loss, acc = trained_global.evaluate(X_val_pad, y_val, verbose=1)
print(f"\nFinal Validation Accuracy: {acc:.4f}")

2216/2216 ━━━━━━━━━━━━━━━━━━━━ 196s 88ms/step - accuracy: 0.7929 - loss: 0.4053

Final Validation Accuracy: 0.7929


Manually adding features

In [134]:
pip install pydot

In [137]:
from tensorflow.keras.utils import plot_model

plot_model(
    trained_global,
    to_file='model_structure.png',
    show_shapes=True,
    show_layer_names=True,
    expand_nested=True,
    dpi=100
)


You must install pydot (`pip install pydot`) for `plot_model` to work.
